In [9]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers.pooling import GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D, LSTM, GRU
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam, SGD
from sklearn.metrics import confusion_matrix
import itertools
from keras.callbacks import ModelCheckpoint
from scipy import stats

In [2]:
import tensorflow as tf

In [5]:
X_train = np.load('18joint_Data/data_nosplit/Weights3/X_train.npy')
X_test = np.load('18joint_Data/data_nosplit/Weights3/X_test.npy')
X_val = np.load('18joint_Data/data_nosplit/Weights3/X_val.npy')
y_test = np.load('18joint_Data/data_nosplit/Weights3/y_test.npy')
y_val = np.load('18joint_Data/data_nosplit/Weights3/y_val.npy')
y_train = np.load('18joint_Data/data_nosplit/Weights3/y_train.npy')

In [21]:
import os
weights_path = os.getcwd()
weights_path

'C:\\Users\\Admin\\Documents\\DataScience_Esha\\CEERI_Intern_PoseDetection'

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 45, 18, 16)        112       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 45, 18, 16)        64        
_________________________________________________________________
time_distributed_3 (TimeDist (None, 45, 18, 16)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 45, 18, 16)        64        
_________________________________________________________________
time_distributed_4 (TimeDist (None, 45, 288)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 45, 20)            24720     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 45, 6)             126       
Total para

In [28]:
def model_CNN_LSTM():
    model = Sequential([
            TimeDistributed(Conv1D(16,3, activation='relu', padding = "same"),input_shape=X_train.shape[1:]),
            TimeDistributed(BatchNormalization()),
            TimeDistributed(Dropout(0.5)),
            BatchNormalization(),
            TimeDistributed(Flatten()),
            LSTM(20,unit_forget_bias = 0.5, return_sequences = True),
            TimeDistributed(Dense(6,activation='softmax'))])
    adam = Adam(lr=0.0001)
    #sgd = SGD(lr=0.0001)
#     Ag = keras.optimizers.Adagrad(lr=0.0001)
#    rms = keras.optimizers.RMSprop(lr=0.0001)
#     lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=1e-4,
#     decay_steps=10000,
#     decay_rate=0.9)
#     rms = keras.optimizers.RMSprop(learning_rate=lr_schedule)
 
    model.compile(loss='categorical_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])
    
    return model

In [38]:
def model_CNN_GRU():
    model = Sequential([
            TimeDistributed(Conv1D(16,3, activation='relu', padding = "same"),input_shape=X_train.shape[1:]),
            TimeDistributed(BatchNormalization()),
            TimeDistributed(Dropout(0.5)),
            BatchNormalization(),
            TimeDistributed(Flatten()),
#             LSTM(20,unit_forget_bias = 0.5, return_sequences = True),
            GRU(20,return_sequences = True),
            TimeDistributed(Dense(6,activation='softmax'))])
    adam = Adam(lr=0.0001)
 
    model.compile(loss='categorical_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])
    
    return model

In [46]:
model = model_CNN_GRU()

In [47]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_35 (TimeDis (None, 45, 18, 16)        112       
_________________________________________________________________
time_distributed_36 (TimeDis (None, 45, 18, 16)        64        
_________________________________________________________________
time_distributed_37 (TimeDis (None, 45, 18, 16)        0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 45, 18, 16)        64        
_________________________________________________________________
time_distributed_38 (TimeDis (None, 45, 288)           0         
_________________________________________________________________
gru_7 (GRU)                  (None, 45, 20)            18540     
_________________________________________________________________
time_distributed_39 (TimeDis (None, 45, 6)             126       
Total para

In [49]:
filepath= weights_path+'\\' + "Weights_GRU\\" + "val1" + "-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model_history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data = (X_val, y_val),callbacks=callbacks_list)

Train on 7686 samples, validate on 2563 samples
Epoch 1/100
7686/7686 [==============================] - 27s 4ms/step - loss: 1.2551 - acc: 0.5620 - val_loss: 0.7035 - val_acc: 0.8176
Epoch 2/100
7686/7686 [==============================] - 24s 3ms/step - loss: 0.6665 - acc: 0.8573 - val_loss: 0.4373 - val_acc: 0.9093
Epoch 3/100
7686/7686 [==============================] - 24s 3ms/step - loss: 0.4465 - acc: 0.9197 - val_loss: 0.3116 - val_acc: 0.9230
Epoch 4/100
7686/7686 [==============================] - 25s 3ms/step - loss: 0.3317 - acc: 0.9353 - val_loss: 0.2454 - val_acc: 0.9297
Epoch 5/100
7686/7686 [==============================] - 23s 3ms/step - loss: 0.2609 - acc: 0.9430 - val_loss: 0.2118 - val_acc: 0.9347
Epoch 6/100
7686/7686 [==============================] - 24s 3ms/step - loss: 0.2171 - acc: 0.9495 - val_loss: 0.1772 - val_acc: 0.9442
Epoch 7/100
7686/7686 [==============================] - 24s 3ms/step - loss: 0.1884 - acc: 0.9557 - val_loss: 0.1660 - val_acc: 0.9463


7686/7686 [==============================] - 24s 3ms/step - loss: 0.0200 - acc: 0.9935 - val_loss: 0.0163 - val_acc: 0.9946
Epoch 49/100
7686/7686 [==============================] - 26s 3ms/step - loss: 0.0201 - acc: 0.9935 - val_loss: 0.0154 - val_acc: 0.9949
Epoch 50/100
7686/7686 [==============================] - 26s 3ms/step - loss: 0.0202 - acc: 0.9936 - val_loss: 0.0173 - val_acc: 0.9945
Epoch 51/100
7686/7686 [==============================] - 26s 3ms/step - loss: 0.0189 - acc: 0.9941 - val_loss: 0.0165 - val_acc: 0.9945
Epoch 52/100
7686/7686 [==============================] - 23s 3ms/step - loss: 0.0191 - acc: 0.9937 - val_loss: 0.0159 - val_acc: 0.9944
Epoch 53/100
7686/7686 [==============================] - 25s 3ms/step - loss: 0.0186 - acc: 0.9939 - val_loss: 0.0161 - val_acc: 0.9946
Epoch 54/100
7686/7686 [==============================] - 24s 3ms/step - loss: 0.0183 - acc: 0.9940 - val_loss: 0.0184 - val_acc: 0.9939
Epoch 55/100
7686/7686 [==============================

In [50]:
model = model_CNN_GRU()
model.load_weights('Weights_GRU/val1-98-0.9959.hdf5')

In [51]:
results = model.evaluate(X_test, y_test)

2562/2562 [==============================] - 3s 1ms/step


In [52]:
print("test loss, test acc:", results)

test loss, test acc: [0.007953946112722831, 0.9969381552762486]
